In [ ]:
!pip install google_play_scraper
import csv
from google_play_scraper import app, reviews,Sort

# ID paket aplikasi yang ingin di-scraper
app_id = 'com.finaccel.android'

# Mengambil informasi aplikasi
app_info = app(app_id)

# Jumlah review yang ingin diambil
total_reviews = 100
reviews_per_request = 200  # Jumlah review per permintaan

# Mengambil review aplikasi sebanyak total_reviews
result = []
continuation_token = None
while len(result) < total_reviews:
    count = min(reviews_per_request, total_reviews - len(result))
    reviews_, continuation_token = reviews(
        app_id,
        lang='id',  # Bahasa review (opsional, default: 'en')
        count=count,  # Jumlah review yang akan diambil (opsional, default: 100)
        sort=Sort.NEWEST,  # Urutan review (opsional, default: 'newest')
        continuation_token=continuation_token
    )
    result.extend(reviews_)

# # Menyimpan hasil scraping dalam file CSV
# csv_file = 'review_aplikasi.csv'
# with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
#     writer = csv.writer(file)
#     writer.writerow(['Rating', 'Tanggal', 'Isi Review'])
#     for review in result:
#         writer.writerow([review['score'], review['date'], review['content']])

# print(f"Data review aplikasi telah disimpan dalam file {csv_file}.")


In [ ]:
from google_play_scraper import app, reviews

# ID paket aplikasi Google Play Store
app_id = 'com.finaccel.android'

# Mendapatkan informasi aplikasi
info = app(app_id)

# Mendapatkan review aplikasi
result = reviews(app_id, lang='id', count=1000)

# Menampilkan review
for review in result:
    print(review['content'])


TypeError: list indices must be integers or slices, not str

In [ ]:
from google_play_scraper import app, reviews_all

# ID aplikasi Kredivoo atau com.finaccel.android di Google Play Store
app_id = 'com.finaccel.android'

# Mengambil informasi aplikasi
app_info = app(app_id)

# Mengambil review aplikasi (maksimum 1000 data)
reviews = reviews_all(
    app_id,
    sleep_milliseconds=0, # Jeda antara setiap permintaan (opsional)
    lang='id', # Bahasa review yang ingin diambil (opsional)
    country='id', # Negara review yang ingin diambil (opsional)

)

# Menyimpan data review ke file CSV
import csv

with open('reviews.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['reviewId', 'userName', 'userImage', 'content', 'score', 'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent', 'repliedAt', 'appVersion']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for review in reviews:
        writer.writerow(review)


In [ ]:
from google_play_scraper import Sort, reviews_all

result = reviews_all(
    'com.bukalapak.android',
    sleep_milliseconds=0, # defaults to 0
    lang='id', # defaults to 'en'
    country='id', # defaults to 'us'
    sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT , you can use Sort.NEWEST to get newst reviews
)

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

def scrape_google_play_reviews(app_id, num_reviews=1000, output_file="reviews.csv"):
    url = f"https://play.google.com/store/apps/details?id={app_id}&hl=id&showAllReviews=true"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    reviews = soup.find_all("div", class_="review-body")
    if len(reviews) > num_reviews:
        reviews = reviews[:num_reviews]

    # Buka file CSV untuk ditulis
    with open(output_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Tulis header baris pertama
        writer.writerow(["Title", "Content", "Rating"])

        for review in reviews:
            title = review.find("div", class_="review-title").text.strip()
            content = review.find("div", class_="review-body-content").text.strip()
            rating = review.find("div", class_="tiny-star").find("div", class_="tiny-star").attrs.get("aria-label")

            # Tulis data review ke dalam file CSV
            writer.writerow([title, content, rating])

# Contoh penggunaan
app_id = "com.finaccel.android" # Ganti dengan ID aplikasi yang Anda inginkan
scrape_google_play_reviews(app_id)


In [ ]:
import requests
from bs4 import BeautifulSoup

def scrape_reviews(app_id, num_reviews=1000):
    reviews = []
    url = f"https://play.google.com/store/apps/details?id={app_id}&hl=id&showAllReviews=true"
    while len(reviews) < num_reviews:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        review_elements = soup.find_all("div", class_="RHo1pe")
        for review in review_elements:
            user = review.find("div", class_="X5PpBb").text
            rating = len(review.find_all("div", class_="iXRFPc"))
            date = review.find("span", class_="bp9Aid").text
            text = review.find("div", class_="h3YV2d").text
            reviews.append({"user": user, "rating": rating, "date": date, "text": text})
            if len(reviews) >= num_reviews:
                break
        next_button = soup.find("span", class_="CwaK9")
        if next_button:
            url = "https://play.google.com" + next_button.parent["href"]
        else:
            break
    return reviews

if __name__ == "__main__":
    app_id = "com.finaccel.android"
    num_reviews = 1000
    reviews = scrape_reviews(app_id, num_reviews)
    for review in reviews:
        print(review)
